# GPT-4o model with Azure OpenAI
<img src="https://github.com/retkowsky/images/blob/master/gpt4o.jpg?raw=true">

GPT-4o is the latest model from OpenAI. GPT-4o integrates text and images in a single model, enabling it to handle multiple data types simultaneously. This multimodal approach enhances accuracy and responsiveness in human-computer interactions. GPT-4o matches GPT-4 Turbo in English text and coding tasks while offering superior performance in non-English languages and vision tasks, setting new benchmarks for AI capabilities.

> Documentation: https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-4o-and-gpt-4-turbo

In [ ]:
import base64
import datetime
import glob
import openai
import os
import requests
import sys

from dotenv import load_dotenv
from io import BytesIO
from mimetypes import guess_type
from openai import AzureOpenAI
from PIL import Image

In [ ]:
def check_openai_version():
    """
    Check Azure Open AI version
    """
    installed_version = openai.__version__

    try:
        version_number = float(installed_version[:3])
    except ValueError:
        print("Invalid OpenAI version format")
        return

    print(f"Installed OpenAI version: {installed_version}")

    if version_number < 1.0:
        print("[Warning] You should upgrade OpenAI to have version >= 1.0.0")
        print("To upgrade, run: %pip install openai --upgrade")
    else:
        print(f"[OK] OpenAI version {installed_version} is >= 1.0.0")

In [ ]:
check_openai_version()

In [ ]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

In [ ]:
print(f"Python version: {sys.version}")

In [ ]:
print(f"OpenAI version: {openai.__version__}")

## 1 Settings

In [ ]:
load_dotenv("azure.env")

# Azure OpenAI
api_type: str = "azure"
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")

api_version = "2024-05-01-preview"

In [ ]:
# This is the deployed name of your GPT-4o model from the Azure Open AI studio
model = "gpt-4o"

## 2 Testing the model

In [ ]:
def gpt4o_text(prompt):
    """
    Gpt-4o model
    """
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        base_url=f"{api_base}/openai/deployments/{model}",
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            },
        ],
        max_tokens=2000,
        temperature=0.7,
    )

    return response

In [ ]:
result = gpt4o_text("Who are you?")
print(result.choices[0].message.content)

In [ ]:
result = gpt4o_text("What is the capital of France?")
print(result.choices[0].message.content)

In [ ]:
result = gpt4o_text("What is an ARMA model?")
print(result.choices[0].message.content)

## 3 Using an url image

In [ ]:
def gpt4o_url(image_url, prompt):
    """
    Gpt-4o model using image url
    """
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        base_url=f"{api_base}/openai/deployments/{model}",
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant to analyse images.",
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": image_url}},
                ],
            },
        ],
        max_tokens=2000,
        temperature=0.0,
    )

    return response

In [ ]:
image_url = "https://github.com/retkowsky/images/blob/master/jo.png?raw=true"

In [ ]:
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
img

In [ ]:
result = gpt4o_url(image_url, "Analyse this image")

print(result.choices[0].message.content)

## 4 Using an image file

In [ ]:
def image_view(image_file):
    """
    View image using PIL
    """
    if not os.path.exists(image_file):
        print(f"[Error] Image file {image_file} does not exist.")
        return None

    else:
        print(image_file)
        img = Image.open(image_file)
        display(img)

In [ ]:
def local_image_to_data_url(image_path):
    """
    Get the url of a local image
    """
    mime_type, _ = guess_type(image_path)

    if mime_type is None:
        mime_type = "application/octet-stream"

    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode("utf-8")

    return f"data:{mime_type};base64,{base64_encoded_data}"

In [ ]:
def gpt4o_imagefile(image_file, prompt):
    """
    Gpt-4o model
    """
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        base_url=f"{api_base}/openai/deployments/{model}",
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant to analyse images.",
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": local_image_to_data_url(image_file)},
                    },
                ],
            },
        ],
        max_tokens=2000,
        temperature=0.0,
    )

    return response

### Example 1

In [ ]:
image_file = "image1.jpg"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(
    image_file, "Extract the main article, its brand and its color."
)
print(result.choices[0].message.content)

In [ ]:
context = """ 
You are a fashion expert, familiar with identifying features of fashion articles from images.
A user uploads an image and asks you to describe one particular piece in the shot: jacket, shoes, pants, \
watches, etc.
"""

prompt = """
You respond with your analysis of the following fields:

1. ITEM'S TYPE: Identify if it's a top, bottom, dress, outerwear, footwear, bag, jewelry...
2. BRAND: identity the brand of the item.
3. COLOR: Note the main color(s) and any secondary colors.
4. PATTERN: Identify any visible patterns such as stripes, florals, animal print, or geometric designs.\
Feel free to use any other patterns here.
5. MATERIAL: Best guess at the material that the item is made from.
6. FEATURES: Note any unique details or embellishments, like embroidery, sequins, studs, fringes, buttons,
zippers...
7. ITEM TYPE SPECIFIC: For each type of item, feel free to add any additional descriptions that are relevant \
to help describe the item. For example, for a jacket you can include the neck and sleeve design, plus the length.
8. MISC.: Anything else important that you notice.
9. SIZE: Print the size of the item if you get it from the image.
10. ITEM SUMMARY: Write a one line summary for this item.
11. ITEM CLASSIFICATION: Classify this item into CLOTHES, BAG, SHOES, WATCH or OTHERS.
12. ITEM TAGS: Generate 10 tags to describe this item. Each tags should be separated with a comma.
13. STORIES: Write multiple stories about this product in 5 lines.
14. TWEETER PUBLICATION: Write a tweeter ad for this item with some hashtags and emojis.
15. ECOMMERCE AD: Generate an item description for a publication on a ecommerce website with a selling message.
16. FRENCH ECOMMERCE AD: Generate an item description in French for a publication on a ecommerce website with \
a selling message.

The output should be a numbered bulleted list. Just print an empty line between each items starting at item 12.
"""

retailprompt = context + prompt

In [ ]:
result = gpt4o_imagefile(image_file, retailprompt)
print(result.choices[0].message.content)

### Example 2

In [ ]:
image_file = "image2.jpg"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(image_file, retailprompt)
print(result.choices[0].message.content)

### Example 3

In [ ]:
image_file = "image3.png"

image_view(image_file)

In [ ]:
context = """
You are a plant and flowers expert, familiar with identifying plants and flowers from images.
A user uploads an image and asks you to describe one particular piece in the shot: flowers, plants...
"""

prompt = """
You respond with your analysis of the following fields:

1. ITEM'S TYPE: Identify the plant species.
2. COLOR: Note the main color(s) and any secondary colors of the plant.
3. COUNTRIES: Identify the country where this plant came from.
4. WATER: Identify the water preferences of this plant.
5. SUN: Identify the light preference of this plant.
6. TEMPERATURE: Identify the best temperature for this plant.
7. FERTILIZATION: What about fertilization?
8. DISEASE: What are the main diseases for this plant?
9. SOIL: What is the best soil?
10. SOIL PH: What is the best soil PH?
11. FLOWERING: What is the flowering period?
12. TOXICITY: What is the toxicity?
13. ITEM TYPE SPECIFIC: For each type of item, feel free to add any additional descriptions that are relevant \
to help describe the item.
14. MISC.: Anything else important that you notice about the plant.
15. SIZE: What is the usual size of this plant?
16. ITEM SUMMARY: Write a one-line summary for this item.
17. ITEM CLASSIFICATION: Classify this item into "plant for beginners", "plant that needs attention", \
"plant for experts".
18. ITEM TAGS: Generate 10 tags to describe this plant. Each tag should be separated with a comma.
19. STORIES: Write multiple stories about this plant in 5 lines.
20. TWEETER PUBLICATION: Write a tweeter ad for this plant with some hashtags and emojis.
21. ECOMMERCE AD: Generate a plant description for a publication on an ecommerce website with a selling message.
22. FRENCH ECOMMERCE AD: Generate a plant description in French for a publication on an ecommerce website with \
a selling message.

The output should be a numbered bulleted list.
"""

flower_prompt = context + prompt

In [ ]:
result = gpt4o_imagefile(image_file, flower_prompt)
print(result.choices[0].message.content)

In [ ]:
result = gpt4o_imagefile(image_file, flower_prompt + " Print all the results in French")
print(result.choices[0].message.content)

### Example 4

In [ ]:
image_file = "image4.jpg"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(
    image_file, "Get the name, dob and the expiration date into a json file format"
)

print(result.choices[0].message.content)

In [ ]:
now = datetime.datetime.now()
current_month = now.strftime("%B")
current_year = now.strftime("%Y")

result = gpt4o_imagefile(
    image_file, f"We are in {current_month} {current_year}. What is the age of Gisèle?"
)

print(result.choices[0].message.content)

In [ ]:
result = gpt4o_imagefile(image_file, "What is the country?")

print(result.choices[0].message.content)

In [ ]:
result = gpt4o_imagefile(
    image_file, "Classify this into PASSPORT, DRIVER LICENCE, ID, TRANSPORTATION CARD"
)

print(result.choices[0].message.content)

### Example 5

In [ ]:
image_file = "image5.jpg"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(image_file, "What is the name and the email?")

print(result.choices[0].message.content)

### Example 6

In [ ]:
image_file = "image6.png"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(
    image_file, "What is the time and the percentage of available memory?"
)

print(result.choices[0].message.content)

In [ ]:
result = gpt4o_imagefile(
    image_file, "Any rain during the week?")
print(result.choices[0].message.content)

### Example 7

In [ ]:
image_file = "image7.jpg"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(image_file, "Describe this")

print(result.choices[0].message.content)

In [ ]:
result = gpt4o_imagefile(
    image_file, "You are a financial analyst. What is your report on this stock?"
)

print(result.choices[0].message.content)

### Example 8

In [ ]:
image_file = "image8.png"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(image_file, "Describe this architecture")

print(result.choices[0].message.content)

In [ ]:
result = gpt4o_imagefile(
    image_file,
    "List the Azure technologies I need to provision into my Azure subscription.",
)

print(result.choices[0].message.content)

### Image 9

In [ ]:
image_file = "image9.jpg"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(image_file, "What is the dimension of the CB line?")

print(result.choices[0].message.content)

In [ ]:
result = gpt4o_imagefile(image_file, "What is the angle between AC and CB")
print(result.choices[0].message.content)

### Image 10

In [ ]:
image_file = "image10.png"

image_view(image_file)

In [ ]:
result = gpt4o_imagefile(image_file, "What's wrong here?")

print(result.choices[0].message.content)

### Image 11

In [ ]:
image_file = "image11.jpg"

image_view(image_file)

In [ ]:
prompt = "Classify this document into 'Driver licence', 'Passport', 'European Accident form', 'Others'"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "What is the language used in this document?"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "Generate a summary."

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "What are the names, cars models of vehicles A and B? Print the results in a json file format."

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "What are the damages for vehicles A and B?"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "Do we have injured people?"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "Do we have some witness?"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "Explain the drawings from section number 10 for vehicles A and B"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "Explain the drawing from section number 13"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)

In [ ]:
prompt = "Do we have signatures on section 15? Just say YES or NO.\
IF the answer is YES, how many signatures do we have? Just print the number"

result = gpt4o_imagefile(image_file, prompt)
print(result.choices[0].message.content)